In [1]:
import os
import sys

project_directory = os.path.abspath('..')
sys.path.append(project_directory)

In [2]:
import torch
import torch.nn as nn
from src.utils.common import set_seed

set_seed(42)

In [3]:
from src.dataloaders.dataloader import create_dataloaders
train_loader, val_loader, test_loader = create_dataloaders(data_root='../data')

In [4]:
train_dataset, val_dataset, test_dataset = create_dataloaders(data_root='../data', return_datasets=True)

In [5]:
from src.models.modern_cnn_v1 import ModernCNNv1

In [6]:
from torch.utils.data import Subset, DataLoader

train_subset = Subset(test_dataset, range(64))
val_subset   = Subset(val_dataset, range(64))

In [7]:
train_loader_small = DataLoader(
    train_subset,
    batch_size=16,
    shuffle=True
)

val_loader_small = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False
)

In [8]:
model = ModernCNNv1()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 20

In [9]:
from torch.optim.lr_scheduler import CosineAnnealingLR

schduler_cosine = CosineAnnealingLR(
    optimizer=optimizer,
    T_max=epochs,
    eta_min=1e-6
)

In [10]:
from src.train.trainer import fit, EarlyStopper

In [11]:
early_stopper = EarlyStopper(patience=10)

test_result = fit(
    model=model,
    train_loader=train_loader_small,
    val_loader=val_loader_small,
    optimizer=optimizer,
    criterion=criterion,
    device=DEVICE, 
    epochs=epochs,
    early_stopper=early_stopper,
    scheduler=schduler_cosine
)

EPOCH 1/20 | Train Loss: 2.880077 Acc: 0.140625 | Val Loss: 2.305816 Acc: 0.095800 
EPOCH 2/20 | Train Loss: 2.294782 Acc: 0.328125 | Val Loss: 2.323969 Acc: 0.122200 
EPOCH 3/20 | Train Loss: 1.948965 Acc: 0.375000 | Val Loss: 2.373700 Acc: 0.104000 
EPOCH 4/20 | Train Loss: 1.823040 Acc: 0.343750 | Val Loss: 2.453759 Acc: 0.114400 
EPOCH 5/20 | Train Loss: 1.603760 Acc: 0.484375 | Val Loss: 2.430076 Acc: 0.144400 
EPOCH 6/20 | Train Loss: 1.415191 Acc: 0.500000 | Val Loss: 2.316049 Acc: 0.184400 
EPOCH 7/20 | Train Loss: 1.115025 Acc: 0.703125 | Val Loss: 2.335671 Acc: 0.194600 
EPOCH 8/20 | Train Loss: 1.158210 Acc: 0.578125 | Val Loss: 2.479532 Acc: 0.166600 
EPOCH 9/20 | Train Loss: 0.958737 Acc: 0.703125 | Val Loss: 2.505361 Acc: 0.155400 
EPOCH 10/20 | Train Loss: 0.777686 Acc: 0.750000 | Val Loss: 2.404414 Acc: 0.195600 
EPOCH 11/20 | Train Loss: 0.680455 Acc: 0.875000 | Val Loss: 2.373309 Acc: 0.216600 
Early stop at epoch 11 | Best epoch 1


In [12]:
test_result

{'train_loss': [2.8800772428512573,
  2.2947816252708435,
  1.9489648342132568,
  1.8230398297309875,
  1.6037601232528687,
  1.4151912331581116,
  1.1150250434875488,
  1.158210352063179,
  0.9587374776601791,
  0.7776860445737839,
  0.6804548352956772],
 'val_loss': [2.3058158504486084,
  2.3239691482543945,
  2.3736998805999754,
  2.453758728027344,
  2.43007551651001,
  2.3160485622406006,
  2.3356714473724365,
  2.4795322967529296,
  2.505361289215088,
  2.4044140048980713,
  2.3733089057922365],
 'train_acc': [0.140625,
  0.328125,
  0.375,
  0.34375,
  0.484375,
  0.5,
  0.703125,
  0.578125,
  0.703125,
  0.75,
  0.875],
 'val_acc': [0.0958,
  0.1222,
  0.104,
  0.1144,
  0.1444,
  0.1844,
  0.1946,
  0.1666,
  0.1554,
  0.1956,
  0.2166]}